# STEP 1: CONNECT TO SQL [POWER PLANT]

In [ ]:
# CONNECT TO SQL

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker, declarative_base

# 1. Define the database file location
# "sqlite:///" tells SQLAlchemy to use SQLite.
# "bookings.db" is the file that will be created in this folder.
DATABASE_URL = "sqlite:///bookings.db"

# 2. Create the Engine
# The "connect_args" part is specific to SQLite to allow
# one thread to talk to the DB while another waits (prevents locking errors).
engine = create_engine(
    DATABASE_URL, 
    connect_args={"check_same_thread": False}
)

# 3. Create the Session Factory
# Think of this as a "dispenser" for database sessions.
# When we need to read/write, we will ask SessionLocal for a session.
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

# 4. Create the Base Class
# All our models (User, Slot, Booking) will inherit from this "Base".
# It allows SQLAlchemy to track them.
Base = declarative_base()

# STEP 2: DATABASE STRUCTURE
## __________ DEFINES TABLES (USERS, SLOT, BOOKING)

In [ ]:
# STEP 2: DATABASE STRUCTURE

from sqlalchemy import Column, Integer, String, DateTime, Boolean, ForeignKey
from sqlalchemy.orm import relationship
from datetime import datetime
from database import Base

# 1. The User Table
class User(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True, index=True)
    username = Column(String, nullable=False)
    email = Column(String, unique=True, index=True, nullable=False)
    role = Column(String, default="student")  # 'student' or 'volunteer'
    created_at = Column(DateTime, default=datetime.utcnow)

    # Relationships
    # "slots" allows us to do user.slots to see what they volunteered for
    slots = relationship("Slot", back_populates="volunteer")
    # "bookings" allows us to do user.bookings to see what they have booked
    bookings = relationship("Booking", back_populates="student")

# 2. The Slot Table (The "Supply")
class Slot(Base):
    __tablename__ = 'slots'

    id = Column(Integer, primary_key=True, index=True)
    volunteer_id = Column(Integer, ForeignKey('users.id'))
    start_time = Column(DateTime, nullable=False)
    end_time = Column(DateTime, nullable=False)
    is_booked = Column(Boolean, default=False)

    # Relationships
    volunteer = relationship("User", back_populates="slots")
    booking = relationship("Booking", back_populates="slot", uselist=False)

# 3. The Booking Table (The "Demand")
class Booking(Base):
    __tablename__ = 'bookings'

    id = Column(Integer, primary_key=True, index=True)
    slot_id = Column(Integer, ForeignKey('slots.id'), unique=True)
    student_id = Column(Integer, ForeignKey('users.id'))
    google_event_id = Column(String, nullable=True) # Stores the ID from Google Calendar
    booked_at = Column(DateTime, default=datetime.utcnow)

    # Relationships
    slot = relationship("Slot", back_populates="booking")
    student = relationship("User", back_populates="bookings")

# STEP 3: Functions to Add/Remove data (CRUD)

In [ ]:
from database import engine, Base
# IMPORTANT: We must import the models here so 'Base' knows they exist!
# If we don't import them, SQLAlchemy won't see the tables we just designed.
from models import User, Slot, Booking

def init_db():
    """
    Creates the database tables based on our models.
    """
    print("Creating database tables...")
    
    # This line is the heavy lifter.
    # It looks at all classes that inherit from 'Base' and creates
    # the SQL tables for them in the engine's database.
    Base.metadata.create_all(bind=engine)
    
    print("Tables created successfully! 'bookings.db' should now exist.")

if __name__ == "__main__":
    # This allows us to run this file directly to setup the DB
    init_db()